# Pods

Like what we discuss before, **Pod** is the smallest and simplest unit in the Kubernetes object model that you create or deploy. It represents one (or sometimes more) running container(s) in a cluster. In a simple web application, you might have one single pod: the web server. As traffic grows, you might deploy that same code to multiple pods to handle the increased load. Several pods, one codebase. In a more complex backend system, you might have several pods for the web server and several pods that handle video processing. Multiple pods, multiple codebases.

Pods are just wrappers around containers. You can think of it as a Docker container with a little extra Kubernetes magic. The container is the actual application, and the Pod is the Kubernetes abstraction that manages the container and the resources it needs to run.

# Basic and First run

We'll be using Kubernetes with Docker, which is arguably the most common way to use Kubernetes. Make sure your Docker daemon is running before starting Minikube.:

```python
minikube start
```

Which will take a few minutes to complete, then run:

```python
minikube dashboard --port=63840
```

This will open a browser window with a locally hosted dashboard for your cluster. You can use this dashboard to view and manage your cluster.

Ok so after finish that we will deploy our first image with kubernetes. Here we use: `kubectl create deployment` command will create a "deployment" for us. We'll talk more about the nuances of "deployments" later. But to put it simply, we only need to provide two things:
- The name of the deployment (this can be anything, it's used to identify the deployment)
- The ID of the Docker image we want to deploy (it would be a full URL if we weren't hosting the image on Docker Hub, which is the default)

```python
kubectl create deployment synergychat-web --image=docker.io/bootdotdev/synergychat-web:latest
```

This command will deploy a container built from [this Docker image](https://hub.docker.com/r/bootdotdev/synergychat-web) to your local k8s cluster.

To make sure the deployment was successful, run:

```python
kubectl get deployments
```

By default, resources inside of Kubernetes run on a private, isolated network. They're visible to other resources within the cluster, but not to the outside world.

In order to access the application from your local network, you'll need to use `kubectl` to do some port forwarding. First, run:

```python
kubectl get pods
```

We'll talk more about pods later, but for now, a pod is an abstraction over a container, and remember, a container is just a running instance of an image. To oversimplify, a pod is a running application.

You should see something like this:

```
NAME                                   READY   STATUS    RESTARTS   AGE
synergychat-web-679cbcc6cd-cq6vx       1/1     Running   0          20m
```

Then run:

```python
kubectl port-forward PODNAME 8080:8080
```

Where `PODNAME` with your pod's name. In my case, it was `synergychat-web-679cbcc6cd-cq6vx`

Next, open your browser and navigate to `http://localhost:8080`. You should see a webpage titled "SynergyChat"! Keep in mind, that we haven't configured all the resources the page needs yet, so the forms won't work, but the page should load.

# Multiple Pod

Let's deploy a second pod!

Use the kubectl get pods again to see a list of all your running pods. You should still only see the one synergychat-web pod. Let's add a second instance!

Run kubectl edit deployment synergychat-web to edit the deployment. This will open the deployment in your default text editor. You should see a big 'ol yaml file. This is the configuration of your deployment. Under the "spec" section, you should see the replicas field set to 1. Change it to 2, save the file, and close the editor.

```
spec:
  ...
  replicas: 2
  ...
```

Run kubectl get pods again. You should see two pods with the command `kubectl get pods`


# Ephemeral

***Pods die, they die often, and sometimes without warning.***

The ephemeral (fancy word for "temporary") nature of Pods is one of the defining features of Kubernetes. Unlike traditional virtual machines (VMs) or physical servers that might run indefinitely (or until hardware failure), Pods are designed to be spun up, torn down, and restarted at a moment's notice.


- **Why are they temporary?** Flexibility and resilience. If a Pod encounters a problem, it can be easily terminated and replaced with a new, healthy instance. This model not only allows for high availability but also promotes immutability. Instead of manually patching or updating existing environments, you spin up new versions of the entire environment.
- **How does it affect me?** As a developer, it's crucial to understand that it's rarely a good idea to store persistent data on a Pod. They can be terminated and replaced, and any locally saved data will be lost. Plan on your image restarting from scratch often!


Let's check how that work:

```python
kubectl get pods
```

Print the logs (what the container is printing to stdout) of your older pod:

```python
kubectl logs PODNAME
```

Kill that older pod (this might take several seconds to complete):

```python
kubectl delete pod PODNAME
```

Get a list of your running pods again:

```python
kubectl get pods
```

Feel free to research more but after delete it, K8s will create a new one and replace the delete (normal behavior)

# Unique IP Addresses

Every Pod in a Kubernetes cluster has a unique internal-to-k8s IP address. By giving each Pod a unique IP, Kubernetes simplifies communication and service discovery within the cluster. Pods within the same Node or across different Nodes can easily communicate.

All the resources inside a k8s cluster are virtualized. So, the IP address of a Pod is not the same as the IP address of the Node it's running on. It's a virtual IP address that is only accessible from within the cluster.

Run this command to get a "wide" output of your pods:

```python
kubectl get pods -o wide
```


It gives a few more columns of information, including the IP address of each Pod. Notice that each Pod has a unique IP address!

Next, run:

```python
kubectl proxy
```

This will start a proxy server on your local machine, probably on `127.0.0.1:8001`. Assuming that's the host, navigate to `http://127.0.0.1:8001/api/v1/namespaces/default/pods` in your browser. You should see a big nasty JSON blob that describes the pods that you have running.